In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w5p1/train.csv
/kaggle/input/2022-ai-w5p1/test.csv
/kaggle/input/2022-ai-w5p1/sample_submit.csv


In [2]:
import random
import torch

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [3]:
train  = pd.read_csv('/kaggle/input/2022-ai-w5p1/train.csv')
test = pd.read_csv('/kaggle/input/2022-ai-w5p1/test.csv')
submit = pd.read_csv('/kaggle/input/2022-ai-w5p1/sample_submit.csv')

In [4]:
np.array(train).shape

(60000, 785)

In [5]:
x_train = np.array(train)[:,1:]
y_train = np.array(train)[:,:1]
x_test = np.array(test)

In [6]:
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)

In [7]:
nb_class = 10
nb_data = len(y_train)
#y_one_hot = torch.zeros(nb_data, nb_class)
#y_one_hot.scatter_(1, y_train, 1)

In [8]:
import torch.optim as optim
import torch.nn.functional as F

In [9]:
W = torch.zeros((784, nb_class), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.000001)
nb_epochs = 1000

for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
    #import pdb;pdb.set_trace()
    # cost 표현번 1번 예시
    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train, 1)
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()


    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()))

Epoch    0/1000 Cost: 2.302585
Epoch  100/1000 Cost: 0.717797
Epoch  200/1000 Cost: 0.559594
Epoch  300/1000 Cost: 0.495502
Epoch  400/1000 Cost: 0.459155
Epoch  500/1000 Cost: 0.435110
Epoch  600/1000 Cost: 0.417725
Epoch  700/1000 Cost: 0.404408
Epoch  800/1000 Cost: 0.393785
Epoch  900/1000 Cost: 0.385053
Epoch 1000/1000 Cost: 0.377708


In [10]:
hypothesis = F.softmax(x_test.matmul(W) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)

In [11]:
submit.columns

Index(['id', 'label'], dtype='object')

In [12]:
submit['label'] = predict
submission=submit.astype(np.str_)
submission.to_csv('submission.csv',mode='w',header=True,index=False)